In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/mobile-price-classification/train.csv
/kaggle/input/mobile-price-classification/test.csv


In [2]:
df = pd.read_csv('/kaggle/input/mobile-price-classification/train.csv')

In [3]:
print('The shape of the data is', df.shape[0] )

The shape of the data is 2000


In [4]:
#Check for null values
(df.isnull().sum()/df.count())*1000

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [5]:
#Reading the columns
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [6]:
#Extracting the features
X = df.drop(['price_range'],1)

In [7]:
#Extracting the label
y = df['price_range'].copy()

In [8]:
#Splitting the train and test data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.3,random_state = 0)

In [9]:
#Initiating a SVM model
from sklearn.svm import LinearSVC
model = LinearSVC(random_state = 0)
#Fitiing the model on the train data
model.fit(X_train,y_train)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=0, tol=0.0001,
          verbose=0)

In [10]:
print('The accuracy of the model is:' ,model.score(X_test,y_test))

The accuracy of the model is: 0.5766666666666667


In [11]:
#Regularization parameters

c_model_1 = LinearSVC(C=4,random_state = 0)
c_model_1.fit(X_train,y_train)
acc_1 = c_model_1.score(X_test,y_test)


c_model_2 = LinearSVC(C=0.01,random_state = 0)
c_model_2.fit(X_train,y_train)
acc_2 = c_model_2.score(X_test,y_test)


c_model_3 = LinearSVC(C=0.005,random_state = 0)
c_model_3.fit(X_train,y_train)
acc_3 = c_model_3.score(X_test,y_test)
print('The accuracy with C=4 is:', acc_1)
print(50*'==')
print('The accuracy with C=0.01 is:', acc_2)
print(50*'==')
print('The accuracy with C=0.005 is:', acc_3)

/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


The accuracy with C=4 is: 0.5766666666666667
The accuracy with C=0.01 is: 0.4583333333333333
The accuracy with C=0.005 is: 0.5316666666666666


/opt/conda/lib/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [12]:
#Trying with different types of Kernels
from sklearn.svm import SVC
poly_model = SVC(kernel = 'poly', random_state = 0)
poly_model.fit(X_train,y_train)
acc_poly = poly_model.score(X_test,y_test)
print('The accuracy of Poly:', acc_poly)

print(50*'=')

from sklearn.svm import SVC
rbf_model = SVC(kernel = 'rbf', random_state = 0)
rbf_model.fit(X_train,y_train)
acc_rbf = rbf_model.score(X_test,y_test)
print('The accuracy of rbf:', acc_poly)

The accuracy of Poly: 0.9533333333333334
The accuracy of rbf: 0.9533333333333334


**One vs All(OVA)**
This is the most simple(and naive) approach. 
In order to classify the four classes, we will construct four different binary classifiers. 
For a given class, the positive examples are all the points in the class, and the negative examples are all the points of the other class.As a result we obtain one binary classifier for each problem.
For every new prediction, we use each classifer and return the classifier which returns a positive result.

Drawback

    Can lead to multiple classes
    Can lead to no class

One heuristic to avoid these ambiguities is to assign it based on the maximum value that the classifier will give out instead of the absolute value(i.e. Instead of just assigning class based on whether the result is positive or negative, assign it based on the actual positive or negative value).

That helps this approach choose the class of the hyperplane that is closest to the example when all classifiers disagree. 
Even after that, this method doesn't always give satisfactory results.

Still, it does remain a popular method for multi-class classification owing to its easy implementation and understandability

In sklearn implementation, LinearSVC implements this strategy by default

In [13]:
#Multi class SVM
# One Vs All
model_ova = SVC(random_state = 0,kernel = 'linear', decision_function_shape = 'ova')
model_ova.fit(X_train,y_train)
acc_ova = model_ova.score(X_test,y_test)
print('Accuracy of One vs all:',acc_ova)


Accuracy of One vs all: 0.9783333333333334


 One vs One

This approach is a upgraded version of OVA, where instead of differentiating one class from all the other classifers, we try to distinguish one class from every other other class.

As a result, we train one classifier per pair of classes, which leads to a total of K(K-1)/2 classifiers for K classes. Predictions are then made using voting. Each new instance is passed to each classifier and the predicted class is recorded. Then the class having the most votes is assigned to the instance.

In SVC, it already comes as a hyper-parameter

class sklearn.svm.SVC(kernel='rbf', C=1.0 , decision_function_shape='ova', random_state=None)

Let's try and implement it to solve our modified phone problem.

In [14]:

#One vs One

model_ovo = SVC(random_state = 0, kernel = 'linear', decision_function_shape = 'ova')
model_ovo.fit(X_train,y_train)
acc_ovo = model_ovo.score(X_test,y_test)
y_pred = model_ovo.predict(X_test)
print('Accuracy of One vs One:', acc_ovo)
print((y_pred).tolist())

Accuracy of One vs One: 0.9783333333333334
[3, 0, 2, 2, 2, 0, 0, 3, 3, 1, 1, 3, 0, 2, 3, 0, 3, 2, 2, 1, 0, 0, 3, 1, 2, 2, 3, 1, 3, 1, 1, 0, 2, 0, 1, 3, 0, 0, 3, 3, 2, 1, 3, 3, 1, 3, 0, 1, 3, 1, 1, 3, 0, 3, 0, 2, 2, 2, 0, 3, 3, 1, 3, 2, 1, 2, 3, 2, 2, 2, 3, 2, 1, 0, 1, 3, 2, 2, 1, 2, 3, 3, 3, 0, 0, 0, 2, 1, 2, 3, 1, 2, 2, 1, 0, 3, 3, 3, 0, 3, 1, 1, 2, 1, 3, 2, 2, 3, 2, 3, 3, 0, 0, 1, 3, 3, 0, 0, 1, 0, 0, 3, 2, 2, 1, 1, 1, 1, 0, 2, 1, 3, 3, 3, 3, 3, 3, 2, 0, 1, 1, 2, 1, 3, 0, 3, 0, 0, 2, 0, 1, 1, 1, 1, 3, 0, 0, 3, 1, 3, 2, 1, 3, 1, 2, 3, 3, 2, 1, 0, 3, 1, 2, 3, 3, 0, 2, 2, 3, 0, 2, 1, 0, 1, 2, 1, 2, 0, 3, 3, 1, 1, 0, 2, 3, 0, 1, 2, 2, 0, 3, 3, 3, 1, 2, 3, 3, 3, 0, 0, 0, 2, 3, 3, 0, 0, 1, 3, 2, 3, 3, 3, 0, 0, 2, 2, 3, 1, 0, 2, 0, 0, 0, 3, 2, 0, 2, 2, 1, 1, 0, 2, 3, 3, 0, 0, 1, 3, 3, 2, 3, 0, 3, 1, 1, 0, 2, 3, 3, 2, 0, 0, 1, 2, 3, 2, 2, 3, 1, 1, 0, 3, 3, 2, 1, 3, 2, 2, 2, 1, 0, 2, 2, 1, 0, 0, 2, 2, 2, 2, 0, 1, 3, 0, 2, 2, 3, 0, 2, 0, 1, 1, 3, 0, 0, 2, 3, 1, 2, 0, 2, 0, 3, 0, 3, 3, 2, 3, 1,

Advantages

   Having the flexibility to choose kernel, SVM works well on a wide range of classification problems, even problems in high dimensions and that are not linearly separable.

   SVM models have generalization(regularisation parameters) in practice, therefore the risk of overfitting is less in SVM.

   SVM works well with even unstructured and semi structured data like text, images and trees.
Disadvantages

   Optimisation of key parameters needs to be done for each problem to get satisfactorily results. Parameters that may result in an excellent classification accuracy for problem A, may result in a poor classification accuracy for problem B.

   Design of Multiclass SVM classifiers(OVA and OVO require multiple SVM instances) are still not optimal enough for practical applications

   The quadratic programming problem(primal formulation) is computationally intensive and with increase in training data it can be very slow on normal machines

   SVM in its pure implementation cannot return a probabilistic confidence value(class probabilities) like another classifier say Logistic Regression. This confidence of prediction is sometimes more important than the actual prediction in many applications

Despite being powerful in theory, it isn't being used extensively today. On unstructured data, it is easily outperformed by neural networks and on structured data, by gradient boosted trees.

Still it's an algorithm worth having in your ML toolset owing to its radical and intuitive approach of solving ML problems.